# Pretrained SBERT(Sentence-transformer) 성능 평가  

In [7]:
import pandas as pd

## 테스트용 문장

In [8]:
sentences = ['제품의 가격이 낮아지고, 품질이 좋아진다. 또한 제품의 다양성이 증가하므로 소비자들은 더 좋은 혜택을 받을 수 있다.', '빵을 더 낮은 값에 살 수 있고, 빵의 맛이 더 좋아져서 소비자들은 많은 혜택을 받는다.']

## Sentence Embedding

In [22]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=ad490e0693a2871a0db421ded416cb69ba190c1eaf6eafb660d7bb54eeb0416e
  Stored in directory: /opt/ml/.cache/pip/wheels/ba/03/bb/7a97840eb54479b328672e15a536e49dc60da200fb21564d53
Successfully built GPUtil


In [24]:
import GPUtil
GPUtil.showUtilization()
#출처: https://memesoo99.tistory.com/53 [미미로그:티스토리]

| ID | GPU | MEM |
------------------
|  0 |  0% |  7% |


In [25]:
# https://wikidocs.net/154530
import numpy as np
import pandas as pd
import urllib.request
import torch
from sentence_transformers import SentenceTransformer

In [26]:
torch.cuda.empty_cache() 

In [17]:
# GPU 캐시 데이터 삭제
model_ml = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

In [27]:
with torch.no_grad():
    embeddings = model_ml.encode(sentences)
sentences, embeddings

(['제품의 가격이 낮아지고, 품질이 좋아진다. 또한 제품의 다양성이 증가하므로 소비자들은 더 좋은 혜택을 받을 수 있다.',
  '빵을 더 낮은 값에 살 수 있고, 빵의 맛이 더 좋아져서 소비자들은 많은 혜택을 받는다.'],
 array([[-0.4455608 , -0.16664696,  0.6133387 , ...,  0.20600976,
         -0.26139683,  0.02431984],
        [-0.23032437,  0.08570942, -0.559556  , ...,  0.04270079,
          0.0989245 ,  0.14554915]], dtype=float32))

In [28]:
sbert_kor = SentenceTransformer('sentence-transformers/xlm-r-large-en-ko-nli-ststb')

Downloading:   0%|          | 0.00/795 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/733 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [29]:
with torch.no_grad():
    embeddings_kor = sbert_kor.encode(sentences)
embeddings_kor

array([[ 0.8155513 , -0.2932835 , -0.21726272, ..., -0.5781419 ,
         0.59006685,  0.7541549 ],
       [ 1.102073  , -0.7120398 , -0.17424865, ..., -0.96989685,
         0.5883625 ,  0.66033745]], dtype=float32)

## 유사도 구하는 함수

In [30]:
from numpy import dot
from numpy.linalg import norm

In [31]:
pairs = []
for i, emb in enumerate(embeddings_kor):
    for j in range(i+1, len(embeddings_kor)):
        pairs.append((sentences[i], sentences[j],emb, embeddings_kor[j]))
#pairs[0]

In [32]:
def get_similarity(ans, right_ans, use="cosine"):
    # Cosine Similarity
    if use == "cosine":
        return dot(ans, right_ans)/(norm(ans)*norm(right_ans))
        
    # Euclidean
    if use == "euclidean":
        if norm(ans-right_ans)==norm(ans-right_ans):
            return norm(ans-right_ans)
        else:
            return -1
    
    
    # Pearson
    if use == "pearson":
        return dot((ans - np.mean(ans)), (right_ans - np.mean(right_ans))) / ((norm(ans - np.mean(ans))) * (norm(right_ans - np.mean(right_ans))))
        

In [33]:
for (sent1,sent2, ans, r_ans) in pairs:
    print(sent1," // ", sent2, get_similarity(ans, r_ans))

제품의 가격이 낮아지고, 품질이 좋아진다. 또한 제품의 다양성이 증가하므로 소비자들은 더 좋은 혜택을 받을 수 있다.  //  빵을 더 낮은 값에 살 수 있고, 빵의 맛이 더 좋아져서 소비자들은 많은 혜택을 받는다. 0.81775975


## Process

### 데이터 1 : validation_v1

In [36]:
df_valid_v1 = pd.read_csv('./validation_v1.csv')
df_valid_v1

,right_ans,student_ans,Label
0,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",1
1,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 나빠진다. 또 제품의 다양성이 감소하고, 소비자들은...",0
2,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",0
3,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 나빠진다. 또 제품의 다양성이 증가하고, 소비자들은...",0
4,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",제과점이 서로 경쟁하면서 가격을 내리거나 품질을 좋게하여 소비자를 모을것인데 그렇게...,1
...,...,...,...
105,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,온도 변화가 갑자기 심해져서,0
106,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,열이 뜨거운 물에서 국자로 옮겨지는데 금속이여서 더 빨리 옮겨진다.,1
107,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,맨처음 열이 뜨거운 물에 있다가 금속을 넣게 되면 열의이동으로 인해 열이 금속 국자...,1
108,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,열이 국자로 전도되었기 때문이다.,1


In [37]:
df_valid_v1.iloc

In [ ]:
prediction = []
for data in df_valid_v1.iloc:
    right_ans = data['right_ans']
    student_ans = data['student_ans']
    label = data['Label']
    #print(right_ans, '//', student_ans, '//', label)
    
    right_ans_emb = sbert_kor.encode([right_ans])
    stu_ans_emb = sbert_kor.encode([student_ans])
    
    sim = get_similarity(right_ans_emb[0], stu_ans_emb[0], use="cosine")
    prediction.append(sim)

In [39]:
df_valid_v1['predict(cosine_sim)'] = prediction
df_valid_v1

,right_ans,student_ans,Label,predict(cosine_sim)
0,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",1,1.000000
1,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 나빠진다. 또 제품의 다양성이 감소하고, 소비자들은...",0,0.595253
2,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",0,0.803344
3,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 나빠진다. 또 제품의 다양성이 증가하고, 소비자들은...",0,0.734047
4,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",제과점이 서로 경쟁하면서 가격을 내리거나 품질을 좋게하여 소비자를 모을것인데 그렇게...,1,0.650092
...,...,...,...,...
105,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,온도 변화가 갑자기 심해져서,0,0.526407
106,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,열이 뜨거운 물에서 국자로 옮겨지는데 금속이여서 더 빨리 옮겨진다.,1,0.910641
107,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,맨처음 열이 뜨거운 물에 있다가 금속을 넣게 되면 열의이동으로 인해 열이 금속 국자...,1,0.888514
108,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,열이 국자로 전도되었기 때문이다.,1,0.646318


In [40]:
def threshhold_num(n = 0.6):
    def threshold(pred):
        if pred > n:
            return 1
        elif pred <=n:
            return 0
    return threshold

In [41]:
df_valid_v1['>0.6'] = df_valid_v1['predict(cosine_sim)'].apply(threshhold_num(0.6))

In [42]:
df_valid_v1['>0.7'] = df_valid_v1['predict(cosine_sim)'].apply(threshhold_num(0.7))

In [43]:
df_valid_v1['>0.8'] = df_valid_v1['predict(cosine_sim)'].apply(threshhold_num(0.8))

In [44]:
df_valid_v1['>0.9'] = df_valid_v1['predict(cosine_sim)'].apply(threshhold_num(0.9))

In [45]:
df_valid_v1

,right_ans,student_ans,Label,predict(cosine_sim),>0.6,>0.7,>0.8,>0.9
0,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",1,1.000000,1,1,1,1
1,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 나빠진다. 또 제품의 다양성이 감소하고, 소비자들은...",0,0.595253,0,0,0,0
2,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",0,0.803344,1,1,1,0
3,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...","제품의 가격이 높아지고, 품질이 나빠진다. 또 제품의 다양성이 증가하고, 소비자들은...",0,0.734047,1,1,0,0
4,"제품의 가격이 낮아지고, 품질이 좋아진다. 또 제품의 다양성이 증가하고, 소비자들은...",제과점이 서로 경쟁하면서 가격을 내리거나 품질을 좋게하여 소비자를 모을것인데 그렇게...,1,0.650092,1,0,0,0
...,...,...,...,...,...,...,...,...
105,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,온도 변화가 갑자기 심해져서,0,0.526407,0,0,0,0
106,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,열이 뜨거운 물에서 국자로 옮겨지는데 금속이여서 더 빨리 옮겨진다.,1,0.910641,1,1,1,1
107,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,맨처음 열이 뜨거운 물에 있다가 금속을 넣게 되면 열의이동으로 인해 열이 금속 국자...,1,0.888514,1,1,1,0
108,금속은 열전도율이 높은 재질이기 때문에 뜨거운 물에서 차가운 금속 국자로 열이 이동...,열이 국자로 전도되었기 때문이다.,1,0.646318,1,0,0,0


In [46]:
df_valid_v1.to_csv('./SBERT_valid_v1_prediction.csv')

## Process 함수화

In [60]:
def process( path, model_name='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'):
    model = SentenceTransformer(model_name)
    df = pd.read_csv(path)
    
    prediction = []
    for data in df.iloc:
        right_ans = data['sent_a']
        student_ans = data['sent_b']
        label = data['labels']
        #print(right_ans, '//', student_ans, '//', label)

        right_ans_emb = model.encode([right_ans])
        stu_ans_emb = model.encode([student_ans])

        sim = get_similarity(right_ans_emb[0], stu_ans_emb[0], use="cosine")
        prediction.append(sim)
        
    df['predict(cosine_sim)'] = prediction

    def threshhold_num(n = 0.6):
        def threshold(pred):
            if pred > n:
                return 1
            elif pred <=n:
                return 0
        return threshold
    
    df['>0.6'] = df['predict(cosine_sim)'].apply(threshhold_num(0.6))
    df['>0.7'] = df['predict(cosine_sim)'].apply(threshhold_num(0.7))
    df['>0.8'] = df['predict(cosine_sim)'].apply(threshhold_num(0.8))
    df['>0.9'] = df['predict(cosine_sim)'].apply(threshhold_num(0.9))
    
    dataset = path.split('/')[-1]
    df.to_csv(f'./results/{model_name.split("/")[1]}_{dataset.split(".")[0]}_prediction.csv')

### 데이터 2: validation_short_v1.csv

In [51]:
process('./validation_short_v1.csv')

### 데이터 3: validation_short_v2.csv

In [53]:
process('./validation_short_v2.csv')

### 데이터 4 : validation_short_wrong_v3.csv

In [54]:
process('./validation_short_wrong_v3.csv')

### 모델 tence-transformers/all-mpnet-base-v2

In [66]:
process('./validation_v1.csv', 'sentence-transformers/all-mpnet-base-v2')

In [68]:
process('./validation_short_v1.csv', 'sentence-transformers/all-mpnet-base-v2')

In [62]:
process('./validation_short_v2.csv', 'sentence-transformers/all-mpnet-base-v2')

In [63]:
process('./validation_short_wrong_v3.csv', 'sentence-transformers/all-mpnet-base-v2')

### 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

In [67]:
process('./validation_v1.csv','sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [69]:
process('./validation_short_v1.csv', 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [70]:
process('./validation_short_v2.csv', 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

### xlm-r-100langs-bert-base-nli-stsb-mean-tokens'

In [71]:
process('./validation_v1.csv','sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

### 'sentence-transformers/all-MiniLM-L6-v2'

In [73]:
process('./validation_v1.csv','sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

### average_word_embeddings_komninos

In [ ]:
process('./validation_v1.csv','sentence-transformers/average_word_embeddings_komninos')